# Language Model

## Aim: To implement language model using LTSM
## Dataset: News article

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.utils import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
import numpy as np

tokenizer = Tokenizer()

def dataset_preparation(data):

	# basic cleanup
	corpus = data.lower().split("\n")

	# tokenization
	tokenizer.fit_on_texts(corpus)
	total_words = len(tokenizer.word_index) + 1

	# create input sequences using list of tokens
	input_sequences = []
	for line in corpus:
		token_list = tokenizer.texts_to_sequences([line])[0]
		for i in range(1, len(token_list)):
			n_gram_sequence = token_list[:i+1]
			input_sequences.append(n_gram_sequence)

	# pad sequences
	max_sequence_len = max([len(x) for x in input_sequences])
	input_sequences = np.array(pad_sequences(input_sequences,
	                                         maxlen=max_sequence_len,
																					 padding='pre'))

	# create predictors and label
	predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
	label = ku.to_categorical(label, num_classes=total_words)

	return predictors, label, max_sequence_len, total_words

def create_model(predictors, label, max_sequence_len, total_words):

	model = Sequential()
	model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
	model.add(LSTM(150, return_sequences = True))
	model.add(Dropout(0.2))
	model.add(LSTM(100,return_sequences=False))
	model.add(Dense(total_words, activation='softmax'))

	model.compile(loss='categorical_crossentropy', optimizer='adam',
	              metrics=['accuracy'])
	earlystop = EarlyStopping(monitor='val_loss', min_delta=0,
	                          patience=5, verbose=0, mode='auto')
	model.fit(predictors, label, epochs=100,validation_split=0.2 ,
	          verbose=1,callbacks=[earlystop])
	print(model.summary())
	return model

def generate_text(seed_text, next_words, max_sequence_len):
	for _ in range(next_words):
		token_list = tokenizer.texts_to_sequences([seed_text])[0]
		token_list = pad_sequences([token_list], maxlen=max_sequence_len-1,
		                           padding='pre')
		# predicted = model.predict_classes(token_list, verbose=0)
		predicted = np.argmax(model.predict(token_list), axis=-1)
		output_word = ""
		for word, index in tokenizer.word_index.items():
			if index == predicted:
				output_word = word
				break
		seed_text += " " + output_word
	return seed_text


In [ ]:
data = open('/content/drive/MyDrive/alice_in_wonderland.txt').read()

predictors, label, max_sequence_len, total_words = dataset_preparation(data)
model = create_model(predictors, label, max_sequence_len, total_words)


Epoch 1/100
627/627 [==============================] - 56s 79ms/step - loss: 6.1740 - accuracy: 0.0525 - val_loss: 6.0191 - val_accuracy: 0.0810
Epoch 2/100
627/627 [==============================] - 55s 87ms/step - loss: 5.7968 - accuracy: 0.0586 - val_loss: 6.0327 - val_accuracy: 0.0845
Epoch 3/100
627/627 [==============================] - 56s 89ms/step - loss: 5.6378 - accuracy: 0.0738 - val_loss: 5.9847 - val_accuracy: 0.0891
Epoch 4/100
627/627 [==============================] - 53s 85ms/step - loss: 5.4912 - accuracy: 0.0778 - val_loss: 5.9613 - val_accuracy: 0.1059
Epoch 5/100
627/627 [==============================] - 53s 85ms/step - loss: 5.3732 - accuracy: 0.0839 - val_loss: 5.9381 - val_accuracy: 0.1049
Epoch 6/100
627/627 [==============================] - 54s 87ms/step - loss: 5.2717 - accuracy: 0.0892 - val_loss: 5.9543 - val_accuracy: 0.0993
Epoch 7/100
627/627 [==============================] - 51s 82ms/step - loss: 5.1893 - accuracy: 0.0935 - val_loss: 5.9828 - val_ac

In [ ]:
print(generate_text("Alice said", 10, max_sequence_len))

1/1 [==============================] - 0s 28ms/step
Alice said to was a little thing ' said the hatter and


## Conclusion: The model stopped at 10 epochs due to earlystopping. It is predicting the next set of words.